In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.metrics import mean_squared_error, make_scorer
import statsmodels.api

In [2]:
auto=pd.read_csv('./data/Auto.csv')

In [3]:
train=auto.sample(n=196, random_state=2)
test=auto[~auto.index.isin(train.index)]
#df[~df.index.isin(boot.index)]

# The Validation Set Approach

In [4]:
lm_1=sm.ols('mpg~horsepower', data=train)
lm_1_mod=lm_1.fit()

In [5]:
pred_1=lm_1_mod.predict(test['horsepower'])

In [6]:
print(f'Linear Model MSE: {mean_squared_error(test["mpg"], pred_1):.2f}')

Linear Model MSE: 25.11


In [7]:
qm=sm.ols('mpg~horsepower+I(horsepower**2)', data=train)
qm_mod=qm.fit()

In [8]:
qm_pred=qm_mod.predict(test['horsepower'])
print(f'Quadratic Model MSE: {mean_squared_error(test["mpg"], qm_pred):.2f}')

Quadratic Model MSE: 19.72


In [9]:
qm=sm.ols('mpg~horsepower+I(horsepower**2)+I(horsepower**3)', data=train)
qm_mod=qm.fit()

qm_pred=qm_mod.predict(test['horsepower'])
print(f'Cubic Model MSE: {mean_squared_error(test["mpg"], qm_pred):.2f}')

Cubic Model MSE: 19.92


# Leave-One-Out Cross Validation

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [11]:
X=statsmodels.api.add_constant(auto['horsepower'])
y=auto['mpg']

C:\Users\Richy\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [12]:
scorer = make_scorer(mean_squared_error)
model=LinearRegression()

LOO=cross_val_score(model, X=X, y=y, cv=X.shape[0], scoring=scorer)

In [13]:
LOO.mean()

24.231513517929226

In [14]:
loo_means=[]
for order in range(1, 6):
    poly = PolynomialFeatures(degree=order,interaction_only=False, include_bias=False)
    
    model=LinearRegression()
    LOO_raised=cross_val_score(model, X=poly.fit_transform(X), y=y, cv=X.shape[0], scoring=scorer)
    
    loo_means.append(LOO_raised.mean())

In [15]:
loo_means

[24.231513517929226,
 19.24821312448975,
 19.334984064043173,
 19.424430310361764,
 19.03321358686367]

# k-Fold Cross-Validation

In [16]:
kf_cv_means=[]
for order in range(1, 11):
    poly = PolynomialFeatures(degree=order,interaction_only=False, include_bias=False)
    
    model=LinearRegression()
    kf_cv_raised=cross_val_score(model, X=poly.fit_transform(X), y=y, cv=10, scoring=scorer)
    
    kf_cv_means.append(kf_cv_raised.mean())

In [17]:
kf_cv_means

[27.439933652339857,
 21.235840055802306,
 21.336606183191904,
 21.353886980908165,
 20.905643195478888,
 20.784472952501865,
 20.880627227790693,
 21.07858300938988,
 21.042177187755662,
 21.027618772515318]

In [18]:
from sklearn.model_selection import KFold

# Use KFold
kf = KFold(n_splits=10, shuffle=True, random_state=1111)

# Create splits
splits = kf.split(X)

# Print the number of indices
for train_index, val_index in splits:
    print("Number of training indices: %s" % len(train_index))
    print("Number of validation indices: %s" % len(val_index))

Number of training indices: 352
Number of validation indices: 40
Number of training indices: 352
Number of validation indices: 40
Number of training indices: 353
Number of validation indices: 39
Number of training indices: 353
Number of validation indices: 39
Number of training indices: 353
Number of validation indices: 39
Number of training indices: 353
Number of validation indices: 39
Number of training indices: 353
Number of validation indices: 39
Number of training indices: 353
Number of validation indices: 39
Number of training indices: 353
Number of validation indices: 39
Number of training indices: 353
Number of validation indices: 39


In [19]:
X=auto['horsepower']
y=auto['mpg']

In [20]:
# Use KFold
kf = KFold(n_splits=10, shuffle=True, random_state=17)

# Create splits
splits = kf.split(X)

kF_CV_means=[]
LR=LinearRegression()

# Access the training and validation indices of splits
for train_index, val_index in splits:
    # Setup the training and validation data
    X_train, y_train = X[train_index], y[train_index]
    X_val, y_val = X[val_index], y[val_index]
    
    #Add constant column
    X_train=statsmodels.api.add_constant(X_train)
    X_val=statsmodels.api.add_constant(X_val)
    
    # Fit the Linear Regression model
    LR.fit(X_train, y_train)
    # Make predictions, and print the accuracy
    predictions = LR.predict(X_val)
    kF_CV_means.append(predictions)
    print("Split accuracy: " + str(mean_squared_error(y_val, predictions)))

Split accuracy: 25.340509269972657
Split accuracy: 20.769461355335974
Split accuracy: 27.341704864673027
Split accuracy: 28.10105011712353
Split accuracy: 15.933777254563257
Split accuracy: 39.79714781695028

C:\Users\Richy\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Richy\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Richy\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Richy\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Richy\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is dep


Split accuracy: 16.901207197517344
Split accuracy: 28.56239675581579
Split accuracy: 17.780969177207037
Split accuracy: 21.29513514653329


C:\Users\Richy\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Richy\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [21]:
kf_cv_means

[27.439933652339857,
 21.235840055802306,
 21.336606183191904,
 21.353886980908165,
 20.905643195478888,
 20.784472952501865,
 20.880627227790693,
 21.07858300938988,
 21.042177187755662,
 21.027618772515318]

# The Bootstrap

In [22]:
port=pd.read_csv('./data/Portfolio.csv', ).drop(['Unnamed: 0'], axis=1)

In [23]:
def alpha(data, num_samples=100):
    indices = np.random.choice(data.index, num_samples, replace=True)
    
    X=data['X'][indices]
    Y=data['Y'][indices]

    return (np.var(Y) - np.cov(X,Y)[0][1])/(np.var(X) + np.var(Y) - 2*np.cov(X,Y)[0][1])

In [24]:
np.random.seed(0)
alpha(port)

0.5603366580074973

In [25]:
samps=1000
np.random.seed(0)

In [26]:
boot_mean=np.array([])
for i in range(samps):
    #np.random.seed(i)
    cov=alpha(port)
    
    boot_mean=np.append(boot_mean, cov)
    

In [27]:
boot_mean

array([0.56033666, 0.65174607, 0.6419836 , 0.5265089 , 0.61294813,
       0.52209904, 0.46775   , 0.75344004, 0.51743963, 0.34067531,
       0.55467457, 0.50726371, 0.45677697, 0.84232309, 0.51437895,
       0.58053411, 0.55137652, 0.61623751, 0.6564123 , 0.45009759,
       0.412622  , 0.48617674, 0.61685721, 0.63181173, 0.5070456 ,
       0.53658192, 0.53840006, 0.61253538, 0.47590297, 0.56768259,
       0.59664443, 0.44780639, 0.59371197, 0.53292039, 0.55115268,
       0.67859663, 0.56698495, 0.52392205, 0.49284052, 0.57410441,
       0.54972   , 0.44702787, 0.58108487, 0.62142922, 0.65780197,
       0.53654144, 0.66946174, 0.40557907, 0.69823427, 0.57634229,
       0.61499109, 0.6551613 , 0.57530644, 0.65694267, 0.48974632,
       0.56829308, 0.56281435, 0.53854187, 0.48431801, 0.67552819,
       0.54836828, 0.76162708, 0.60517566, 0.82613741, 0.54471018,
       0.71617603, 0.52345425, 0.68945531, 0.47532653, 0.55894233,
       0.5041125 , 0.46425938, 0.53494607, 0.77037703, 0.57043

In [28]:
print(f'Bootstrap Statistics:\n\
Mean Alpha: {boot_mean.mean():.4}\n\
Std. Error: {boot_mean.std():.4}')

Bootstrap Statistics:
Mean Alpha: 0.5806
Std. Error: 0.0898


In [32]:
data=auto.sample(n=len(auto.index), replace=True, random_state=1)

In [40]:
LM=sm.ols('mpg~horsepower', data=data).fit()

In [41]:
print(LM.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     581.8
Date:                Wed, 29 Apr 2020   Prob (F-statistic):           2.52e-79
Time:                        16:10:56   Log-Likelihood:                -1174.0
No. Observations:                 392   AIC:                             2352.
Df Residuals:                     390   BIC:                             2360.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.6585      0.714     55.575      0.0

In [50]:
params=LM.params
print(f'Intercept: {params[0]:.4}\nhorsepower: {params[1]:.4}')

Intercept: 39.66
horsepower: -0.1559


### Will Get Different Outcomes Depending On Sample

In [51]:
data=auto.sample(n=len(auto.index), replace=True, random_state=10)
LM=sm.ols('mpg~horsepower', data=data).fit()
params=LM.params
print(f'Intercept: {params[0]:.4}\nhorsepower: {params[1]:.4}')

Intercept: 40.45
horsepower: -0.1614


In [71]:
inter_means=[]
horses_means=[]

inter_std=[]
horses_std=[]
boots=1000
for i in range(boots):
    data=auto.sample(n=len(auto.index), replace=True, random_state=1)
    LM=sm.ols('mpg~horsepower', data=data).fit()
    
    inter_means.append(LM.params[0])
    horses_means.append(LM.params[1])
    
    inter_std.append(LM.bse[0])
    horses_std.append(LM.bse[1])

In [72]:
print(f'Intercept: {np.mean(inter_means):.4}    Std. Error: {np.mean(inter_std):.4}\nhorsepower: {np.mean(horses_means):.4}    Std. Error: {np.mean(horses_std):.4}')

Intercept: 39.66    Std. Error: 0.7136
horsepower: -0.1559    Std. Error: 0.006464


In [54]:
LM.params

Intercept     39.658479
horsepower    -0.155898
dtype: float64

In [67]:
LM.bse

Intercept     0.713597
horsepower    0.006464
dtype: float64

In [74]:
inter_means=[]
horses_means=[]
horses2_means=[]

inter_std=[]
horses_std=[]
horses2_std=[]
boots=1000
for i in range(boots):
    data=auto.sample(n=len(auto.index), replace=True, random_state=1)
    LM=sm.ols('mpg~horsepower+I(horsepower**2)', data=data).fit()
    
    inter_means.append(LM.params[0])
    horses_means.append(LM.params[1])
    horses2_means.append(LM.params[2])
    
    inter_std.append(LM.bse[0])
    horses_std.append(LM.bse[1])
    horses2_std.append(LM.bse[2])

In [75]:
print(f'Intercept: {np.mean(inter_means):.4}    Std. Error: {np.mean(inter_std):.4}\n\
horsepower: {np.mean(horses_means):.4}    Std. Error: {np.mean(horses_std):.4}\n\
horsepower^2: {np.mean(horses2_means):.4}    Std. Error: {np.mean(horses2_std):.4}')

Intercept: 57.59    Std. Error: 1.802
horsepower: -0.4776    Std. Error: 0.03082
horsepower^2: 0.001266    Std. Error: 0.0001192
